In [20]:
import os
import pickle
from scipy.io import loadmat
import numpy as np
import scipy as sp 
import scipy.linalg as la
import sklearn.discriminant_analysis as lda
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import cohen_kappa_score
import mne
from util import *
from pathlib import Path
from sklearn.metrics import accuracy_score


In [2]:
def csp_train(train_data_1, train_data_2, numFilt):

    numTrials_1 = np.size(train_data_1, 0)
    numTrials_2 = np.size(train_data_2, 0)

    # csp_train the CSP filters (generalized eigenvalue problem)
    csp_filts = calc_CSP(train_data_1, train_data_2, numFilt)

    # extract the features
    train_filt_1 = apply_CSP(train_data_1, csp_filts, numFilt)
    train_logP_1 = np.squeeze(np.log(np.var(train_filt_1, axis=2)))

    train_filt_2 = apply_CSP(train_data_2, csp_filts, numFilt)
    train_logP_2 = np.squeeze(np.log(np.var(train_filt_2, axis=2)))

    # define the classifier
    clf = lda.LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

    # stack features for both classes
    X = np.concatenate((train_logP_1, train_logP_2), axis=0)
    y1 = np.zeros(numTrials_1)
    y2 = np.ones(numTrials_2)
    y = np.concatenate((y1, y2))

    # csp_train the classifier
    clf.fit(X, y)

    return csp_filts, clf


def csp_test(test_data, csp_filts, clf):
    total_filts = csp_filts.shape[1]
    num_trials = test_data.shape[0]
    
    filtered_data = np.zeros((num_trials, total_filts))
    for trial in range(num_trials):
        trial_data = test_data[trial, :, :]  # Extract single trial (channels, samples)
        # Apply spatial filters
        trial_filtered = np.dot(csp_filts.T, trial_data)
        # Calculate log variance for the trial
        log_var = np.log(np.var(trial_filtered, axis=1))
        filtered_data[trial, :] = log_var
        
    # Classify using clf, get probability of class 0
    return clf.predict_proba(filtered_data)[:, 0]



def calc_CSP(x1, x2, numFilt):
    """
    Standard CSP using generalized eigenvalue decomposition.
    """
    num_trials_1 = np.size(x1, 0)
    num_trials_2 = np.size(x2, 0)
    n_chans = np.size(x1, 1)

    # Compute average trace-normalized covariance for class 1
    c1 = np.zeros((n_chans, n_chans))
    for i in range(num_trials_1):
        cov_i = np.cov(x1[i])
        cov_i /= (np.trace(cov_i) + 1e-12)
        c1 += cov_i
    c1 /= num_trials_1

    # Compute average covariance for class 2
    c2 = np.zeros((n_chans, n_chans))
    for i in range(num_trials_2):
        cov_i = np.cov(x2[i])
        cov_i /= (np.trace(cov_i) + 1e-12)
        c2 += cov_i
    c2 /= num_trials_2

    # Solve the generalized eigenvalue problem
    R = c1 + c2
    eigvals, eigvecs = sp.linalg.eigh(c1, R)

    # Sort eigenvectors by eigenvalues in descending order
    idx = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, idx]

    # select filters
    filts_1 = eigvecs[:, :numFilt]
    filts_1 = select_filts(filts_1, numFilt)

    filts_2 = eigvecs[:, -numFilt:]
    filts_2 = select_filts(filts_2, numFilt)

    return np.concatenate((filts_1, filts_2), axis=1)


def select_filts(filt, col_num):
    temp = np.shape(filt)
    columns = np.arange(0, col_num)
    f = filt[:, columns].copy()
    for ij in range(col_num):
        f[:, ij] /= np.linalg.norm(f[:, ij]) + 1e-12
    return f


def apply_CSP(X, f, col_num):
    f = np.transpose(f)
    temp = np.shape(X)
    num_trials = temp[0]

    dat = np.zeros((num_trials, 2*col_num, temp[2]))
    for ik in range(num_trials):
        dat[ik, :, :] = np.matmul(f, X[ik, :, :])
    return dat

In [3]:
n_sample = 72
k = 6
bin_size = int(n_sample / k)
#n_channel, n_time, n_sample = EEGR_train[0][0].shape
randIdx = np.random.permutation(n_sample)
randIdx = randIdx.reshape(k, bin_size)

In [4]:
passbands = [
    (1, 3), (2, 5), (4, 7), (6, 10), (7, 12),
    (10, 15), (12, 19), (15, 25), (19, 30),
    (25, 35), (30, 40)
]

gdf_folder = Path("BCICIV_2a_gdf")

target_events = {'769': 'left', '770': 'right', '771': 'feet', '772': 'tongue'}

filtered_data = []
labels_2d = []

# skipping AO4T because only has eye movement
users = ["A01T.gdf", "A02T.gdf", "A03T.gdf", "A05T.gdf", "A06T.gdf", "A07T.gdf", "A08T.gdf", "A09T.gdf"]

for subject in users:
    # Iterate through users
    gdf_path = gdf_folder / subject
    print(f"\nProcessing {subject}...")
    
    raw = mne.io.read_raw_gdf(gdf_path.resolve(), preload=True)
    events, event_id_map = mne.events_from_annotations(raw)

    subject_data = []
    user_labels = []

    user_event_ids = {target_events[key]: event_id_map[key] for key in target_events if key in event_id_map}

    # Filter and epoch for each band
    for band in passbands:
        low, high = band
        print(f"  Filtering {low}-{high} Hz...")

        # Bandpass filter
        raw_filtered = raw.copy().filter(l_freq=low, h_freq=high, method='fir', fir_design='firwin')

        # Create epochs
        epochs = mne.Epochs(raw_filtered, events, event_id=user_event_ids, baseline=(None, 0), event_repeated='merge', preload=True, tmin=-.3, tmax=4)
        labels = epochs.events[:, -1]  # Last column contains the event ID

        # Collect data and labels
        subject_data.append(epochs.get_data()[:, :-3, :])  # Shape: (n_trials, n_channels, n_times)
        user_labels.append(labels)

    filtered_data.append(subject_data)
    labels_2d.append(user_labels)


Processing A01T.gdf...
Extracting EDF parameters from F:\COGS189-BCI-Competition-Project\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


f:\Anaconda\envs\CSE156\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
  Filtering 1-3 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 3.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 2-5 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 5.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Filter length: 413 samples (1.652 s)

Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...


...

0 bad epochs dropped
  Filtering 25-35 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 133 samples (0.532 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 30-40 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 111 samples (0.444 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped


In [5]:
data_left = []
data_right = []
data_feet = []
data_tongue = []

for subject in range(len(filtered_data)):
    subject_data_left = []
    subject_data_right = []
    subject_data_feet = []
    subject_data_tongue = []
    for band in range(len(filtered_data[subject])):
        band_data_left = []
        band_data_right = []
        band_data_feet = []
        band_data_tongue = []
        for label in range(len(labels_2d[subject][band])):
            if labels_2d[subject][band][label] == 7:
                band_data_left.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 8:
                band_data_right.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 9:
                band_data_feet.append(filtered_data[subject][band][label])
            else:
                band_data_tongue.append(filtered_data[subject][band][label])
        subject_data_left.append(band_data_left)
        subject_data_right.append(band_data_right)
        subject_data_feet.append(band_data_feet)
        subject_data_tongue.append(band_data_tongue)
    data_left.append(subject_data_left)
    data_right.append(subject_data_right)
    data_feet.append(subject_data_feet)
    data_tongue.append(subject_data_tongue)

data_left = np.array(data_left)
data_right = np.array(data_right)
data_feet = np.array(data_feet)
data_tongue = np.array(data_tongue)

data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076))

In [6]:
data_OVR_left = []
data_OVR_right = []
data_OVR_feet = []
data_OVR_tongue = []

random_samples = np.random.permutation(72)
random_sample_range1 = random_samples[:24]
random_sample_range2 = random_samples[24:48]
random_sample_range3 = random_samples[48:72]

# data_OVR_left
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_left.append(subject_data)

# data_OVR_right
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_right.append(subject_data)

# data_OVR_feet
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_feet.append(subject_data)

# data_OVR_tongue
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_feet[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_tongue.append(subject_data)

# Convert to numpy arrays
data_OVR_left = np.array(data_OVR_left)
data_OVR_right = np.array(data_OVR_right)
data_OVR_feet = np.array(data_OVR_feet)
data_OVR_tongue = np.array(data_OVR_tongue)

In [14]:
# # Reordering dimensions
data_left = data_left.transpose(0, 1, 3, 4, 2)
data_right = data_right.transpose(0, 1, 3, 4, 2)
data_feet = data_feet.transpose(0, 1, 3, 4, 2)
data_tongue = data_tongue.transpose(0, 1, 3, 4, 2)

data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72))

In [7]:
# # Reordering dimensions
data_OVR_left = data_OVR_left.transpose(0, 1, 3, 4, 2)
data_OVR_right = data_OVR_right.transpose(0, 1, 3, 4, 2)
data_OVR_feet = data_OVR_feet.transpose(0, 1, 3, 4, 2)
data_OVR_tongue = data_OVR_tongue.transpose(0, 1, 3, 4, 2)

data_OVR_left.shape, data_OVR_right.shape, data_OVR_feet.shape, data_OVR_tongue.shape

((8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72))

In [8]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline

import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery


moabb.set_log_level("info")
warnings.filterwarnings("ignore")

dataset = BNCI2014_001()

sessions = dataset.get_data()

In [9]:
passbands = [
    (1, 3), (2, 5), (4, 7), (6, 10), (7, 12),
    (10, 15), (12, 19), (15, 25), (19, 30),
    (25, 35), (30, 40)
]

filtered_data = []
labels_2d = []

# skipping AO4T because only has eye movement
runs = ["0", "1", "2", "3", "4", "5"]

skip = 4

for subject in range(1, 10):
    print("Subject: ", subject)
    if subject == skip:
        continue

    # List to store all raw objects
    raw_list = []
    
    # Iterate over runs and add each raw to the list
    for run in runs:
        raw = sessions[subject]["1test"][run]
        raw_list.append(raw)
    
    # Concatenate all raw objects into one
    combined_raw = mne.concatenate_raws(raw_list)
    
    # Iterate through users
    
    raw = combined_raw
    events, event_id = mne.events_from_annotations(raw)

    subject_data = []
    user_labels = []

    # Filter and epoch for each band
    for band in passbands:
        low, high = band

        # Bandpass filter
        raw_filtered = raw.copy().filter(l_freq=low, h_freq=high, method='fir', fir_design='firwin')

        # Create epochs
        epochs = mne.Epochs(raw_filtered, events, event_id=event_id, baseline=(None, 0), event_repeated='merge', preload=True, tmin=-.3, tmax=4)
        labels = epochs.events[:, -1]  # Last column contains the event ID

        # Collect data and labels
        subject_data.append(epochs.get_data()[:, :-4, :])  # Shape: (n_trials, n_channels, n_times)
        user_labels.append(labels)
    filtered_data.append(subject_data)
    labels_2d.append(user_labels)

Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9


In [10]:
test_data_left = []
test_data_right = []
test_data_feet = []
test_data_tongue = []

for subject in range(len(filtered_data)):
    subject_data_left = []
    subject_data_right = []
    subject_data_feet = []
    subject_data_tongue = []
    for band in range(len(filtered_data[subject])):
        band_data_left = []
        band_data_right = []
        band_data_feet = []
        band_data_tongue = []
        for label in range(len(labels_2d[subject][band])):
            if labels_2d[subject][band][label] == 2:
                band_data_left.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 3:
                band_data_right.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 1:
                band_data_feet.append(filtered_data[subject][band][label])
            else:
                band_data_tongue.append(filtered_data[subject][band][label])
        subject_data_left.append(band_data_left)
        subject_data_right.append(band_data_right)
        subject_data_feet.append(band_data_feet)
        subject_data_tongue.append(band_data_tongue)
    test_data_left.append(subject_data_left)
    test_data_right.append(subject_data_right)
    test_data_feet.append(subject_data_feet)
    test_data_tongue.append(subject_data_tongue)

test_data_left = np.array(test_data_left)
test_data_right = np.array(test_data_right)
test_data_feet = np.array(test_data_feet)
test_data_tongue = np.array(test_data_tongue)

test_data_left.shape, test_data_right.shape, test_data_feet.shape, test_data_tongue.shape

((8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076))

In [11]:
test_data_OVR_left = []
test_data_OVR_right = []
test_data_OVR_feet = []
test_data_OVR_tongue = []

random_samples = np.random.permutation(72)
random_sample_range1 = random_samples[:24]
random_sample_range2 = random_samples[24:48]
random_sample_range3 = random_samples[48:72]

# data_OVR_left
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_right[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_left.append(subject_data)

# data_OVR_right
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_right.append(subject_data)

# data_OVR_feet
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_feet.append(subject_data)

# data_OVR_tongue
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_feet[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_tongue.append(subject_data)

# Convert to numpy arrays
test_data_OVR_left = np.array(test_data_OVR_left)
test_data_OVR_right = np.array(test_data_OVR_right)
test_data_OVR_feet = np.array(test_data_OVR_feet)
test_data_OVR_tongue = np.array(test_data_OVR_tongue)

test_data_OVR_left = test_data_OVR_left.transpose(0, 1, 3, 4, 2)
test_data_OVR_right = test_data_OVR_right.transpose(0, 1, 3, 4, 2)
test_data_OVR_feet = test_data_OVR_feet.transpose(0, 1, 3, 4, 2)
test_data_OVR_tongue = test_data_OVR_tongue.transpose(0, 1, 3, 4, 2)

test_data_left = test_data_left.transpose(0, 1, 3, 4, 2)
test_data_right = test_data_right.transpose(0, 1, 3, 4, 2)
test_data_feet = test_data_feet.transpose(0, 1, 3, 4, 2)
test_data_tongue = test_data_tongue.transpose(0, 1, 3, 4, 2)

In [13]:
test_data_left.shape

(8, 11, 22, 1076, 72)

## Left vs Rest

In [15]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            L_train = data_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            L_val = data_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((L_train, OVR_train), axis=0)
            val_data = np.stack((L_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Right vs Rest

In [16]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            r_train = data_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            r_val = data_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((r_train, OVR_train), axis=0)
            val_data = np.stack((r_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Feet vs Rest

In [17]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            f_train = data_feet[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_feet[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            f_val = data_feet[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_feet[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((f_train, OVR_train), axis=0)
            val_data = np.stack((f_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Tongue vs Rest

In [18]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Declare useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
csp_per_class = 3

# Initialize accuracy and kappa arrays
accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        print(f'Processing Fold {fold+1}/{k}')

        # Initialize band scores separately for CCACSP and CSP
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))

        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data
            t_train = data_tongue[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_tongue[subject, band][:,:,trainIdx[fold]].transpose(2,0,1)
            t_val = data_tongue[subject, band][:,:,valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_tongue[subject, band][:,:,valIdx[fold]].transpose(2,0,1)

            train_data = np.stack((t_train, OVR_train), axis=0)
            val_data = np.stack((t_val, OVR_val), axis=0)

            # CCACSP
            ccacsp_filts, clf_CCACSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)

            band_score_train_CCACSP[:, band] = train_prob_CCACSP
            band_score_val_CCACSP[:, band] = val_prob_CCACSP

            # CSP 
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)


            band_score_train_CSP[:, band] = train_prob_CSP
            band_score_val_CSP[:, band] = val_prob_CSP


        # True labels
        y_train = np.append(np.ones(train_size), np.zeros(train_size))
        y_val = np.append(np.ones(val_size), np.zeros(val_size))

        # CCACSP metrics
        y_pred_train_CCACSP = np.rint(band_score_train_CCACSP.mean(1))
        y_pred_val_CCACSP = np.rint(band_score_val_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = sum(y_pred_val_CCACSP == y_val)/len(y_val)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_pred_val_CCACSP, y_val)

        # CSP metrics
        y_pred_train_CSP = np.rint(band_score_train_CSP.mean(1))
        y_pred_val_CSP = np.rint(band_score_val_CSP.mean(1))
        accuracy_CSP[subject, fold] = sum(y_pred_val_CSP == y_val)/len(y_val)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_pred_val_CSP, y_val)

    print('===============')

# Final Output
print("CCACSP mean accuracy per subject:", np.mean(accuracy_CCACSP, axis=1))
print("CCACSP mean kappa per subject:", np.mean(kappa_CCACSP, axis=1))
print("CSP mean accuracy per subject:", np.mean(accuracy_CSP, axis=1))
print("CSP mean kappa per subject:", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Left vs Rest - Test Data

In [21]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            l_train = data_left[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_left[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            l_val = data_left[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_left[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            l_test = test_data_left[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_left[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((l_train, OVR_train), axis=0)
            val_data = np.stack((l_val, OVR_val), axis=0)
            test_data = np.stack((l_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf_CCACSP)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf_CSP)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Subject 1 done.
Processing Subject 2/8
Subject 2 done.
Processing Subject 3/8
Subject 3 done.
Processing Subject 4/8
Subject 4 done.
Processing Subject 5/8
Subject 5 done.
Processing Subject 6/8
Subject 6 done.
Processing Subject 7/8
Subject 7 done.
Processing Subject 8/8
Subject 8 done.
Mean Accuracy (CCACSP): [0.91898148 0.69212963 0.82523148 0.63657407 0.6724537  0.85763889
 0.89699074 0.9849537 ]
Mean Kappa (CCACSP): [0.83796296 0.38425926 0.65046296 0.27314815 0.34490741 0.71527778
 0.79398148 0.96990741]
Mean Accuracy (Classic CSP): [0.91898148 0.69212963 0.82523148 0.63657407 0.6724537  0.85763889
 0.89699074 0.9849537 ]
Mean Kappa (CSP): [0.83796296 0.38425926 0.65046296 0.27314815 0.34490741 0.71527778
 0.79398148 0.96990741]


## Right vs Rest - Test Data

In [22]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            r_train = data_right[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_right[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            r_val = data_right[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_right[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            r_test = test_data_right[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_right[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((r_train, OVR_train), axis=0)
            val_data = np.stack((r_val, OVR_val), axis=0)
            test_data = np.stack((r_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf_CCACSP)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf_CSP)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Subject 1 done.
Processing Subject 2/8
Subject 2 done.
Processing Subject 3/8
Subject 3 done.
Processing Subject 4/8
Subject 4 done.
Processing Subject 5/8
Subject 5 done.
Processing Subject 6/8
Subject 6 done.
Processing Subject 7/8
Subject 7 done.
Processing Subject 8/8
Subject 8 done.
Mean Accuracy (CCACSP): [0.86921296 0.66435185 0.96180556 0.76388889 0.66782407 0.93981481
 0.87962963 0.6724537 ]
Mean Kappa (CCACSP): [0.73842593 0.3287037  0.92361111 0.52777778 0.33564815 0.87962963
 0.75925926 0.34490741]
Mean Accuracy (Classic CSP): [0.87268519 0.68055556 0.9525463  0.75347222 0.6712963  0.93865741
 0.88541667 0.58796296]
Mean Kappa (CSP): [0.74537037 0.36111111 0.90509259 0.50694444 0.34259259 0.87731481
 0.77083333 0.17592593]


## Feet vs Rest - Test Data

In [23]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            f_train = data_feet[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_feet[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            f_val = data_feet[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_feet[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            f_test = test_data_feet[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_feet[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((f_train, OVR_train), axis=0)
            val_data = np.stack((f_val, OVR_val), axis=0)
            test_data = np.stack((f_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf_CCACSP)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf_CSP)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Subject 1 done.
Processing Subject 2/8
Subject 2 done.
Processing Subject 3/8
Subject 3 done.
Processing Subject 4/8
Subject 4 done.
Processing Subject 5/8
Subject 5 done.
Processing Subject 6/8
Subject 6 done.
Processing Subject 7/8
Subject 7 done.
Processing Subject 8/8
Subject 8 done.
Mean Accuracy (CCACSP): [0.8275463  0.84490741 0.78009259 0.55324074 0.71412037 0.84259259
 0.78819444 0.77199074]
Mean Kappa (CCACSP): [0.65509259 0.68981481 0.56018519 0.10648148 0.42824074 0.68518519
 0.57638889 0.54398148]
Mean Accuracy (Classic CSP): [0.80439815 0.83564815 0.77777778 0.59259259 0.72685185 0.84375
 0.75810185 0.82638889]
Mean Kappa (CSP): [0.6087963  0.6712963  0.55555556 0.18518519 0.4537037  0.6875
 0.5162037  0.65277778]


## Tongue vs Rest -Test Data

In [26]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))
# Useful variables
n_subjects, n_filters = 8, 11
train_size = bin_size * (k - 1)
val_size = bin_size
test_size = 72
csp_per_class = 3

accuracy_CCACSP = np.zeros((n_subjects, k))
kappa_CCACSP = np.zeros((n_subjects, k))

accuracy_CSP = np.zeros((n_subjects, k))
kappa_CSP = np.zeros((n_subjects, k))

print('===============')
for subject in range(n_subjects):
    print(f'Processing Subject {subject+1}/{n_subjects}')
    for fold in range(k):
        band_score_train_CCACSP = np.zeros((train_size*2, n_filters))
        band_score_val_CCACSP = np.zeros((val_size*2, n_filters))
        band_score_test_CCACSP = np.zeros((test_size*2, n_filters))
        band_score_train_CSP = np.zeros((train_size*2, n_filters))
        band_score_val_CSP = np.zeros((val_size*2, n_filters))
        band_score_test_CSP = np.zeros((test_size*2, n_filters))

        for band in range(n_filters):
            # Prepare the data as in your original script
            t_train = data_tongue[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            OVR_train = data_OVR_tongue[subject, band][:, :, trainIdx[fold]].transpose(2,0,1)
            
            t_val = data_tongue[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            OVR_val = data_OVR_tongue[subject, band][:, :, valIdx[fold]].transpose(2,0,1)
            
            t_test = test_data_tongue[subject, band][:,:,:].transpose(2,0,1)
            OVR_test = test_data_OVR_tongue[subject, band][:,:,:].transpose(2,0,1)

            train_data = np.stack((t_train, OVR_train), axis=0)
            val_data = np.stack((t_val, OVR_val), axis=0)
            test_data = np.stack((t_test, OVR_test), axis=0)

            # Run CCACSP
            ccacsp_filts, clf_CCACSP = train(train_data[0], train_data[1], csp_per_class)
            train_prob_CCACSP = test(np.vstack(train_data), ccacsp_filts, clf_CCACSP)
            val_prob_CCACSP = test(np.vstack(val_data), ccacsp_filts, clf_CCACSP)
            test_prob_CCACSP = test(np.vstack(test_data), ccacsp_filts, clf_CCACSP)

            # Run CSP
            csp_filts, clf_CSP = csp_train(train_data[0], train_data[1], csp_per_class)
            train_prob_CSP = csp_test(np.vstack(train_data), csp_filts, clf_CSP)
            val_prob_CSP = csp_test(np.vstack(val_data), csp_filts, clf_CSP)
            test_prob_CSP = csp_test(np.vstack(test_data), csp_filts, clf_CSP)

            # Stack probabilities for each band
            band_score_train_CCACSP[:,band] = train_prob_CCACSP
            band_score_val_CCACSP[:,band] = val_prob_CCACSP
            band_score_test_CCACSP[:, band] = test_prob_CCACSP
            band_score_train_CSP[:,band] = train_prob_CSP
            band_score_val_CSP[:,band] = val_prob_CSP
            band_score_test_CSP[:, band] = test_prob_CSP

        # True labels
        y_true = np.append(np.ones(72), np.zeros(72))

        # CCACSP predictions & accuracy/kappa
        y_pred_CCACSP = np.rint(band_score_test_CCACSP.mean(1))
        accuracy_CCACSP[subject, fold] = accuracy_score(y_true, y_pred_CCACSP)
        kappa_CCACSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CCACSP)

        # CSP predictions and accuracy
        y_pred_CSP = np.rint(band_score_test_CSP.mean(1))
        accuracy_CSP[subject, fold] = accuracy_score(y_true, y_pred_CSP)
        kappa_CSP[subject, fold] = cohen_kappa_score(y_true, y_pred_CSP)

    print(f'Subject {subject+1} done.')
    print('===============')

# Final Mean Accuracies and Kappas for each method
print("Mean Accuracy (CCACSP):", np.mean(accuracy_CCACSP, axis=1))
print("Mean Kappa (CCACSP):", np.mean(kappa_CCACSP, axis=1))
print("Mean Accuracy (Classic CSP):", np.mean(accuracy_CSP, axis=1))
print("Mean Kappa (CSP):", np.mean(kappa_CSP, axis=1))

Processing Subject 1/8
Subject 1 done.
Processing Subject 2/8
Subject 2 done.
Processing Subject 3/8
Subject 3 done.
Processing Subject 4/8
Subject 4 done.
Processing Subject 5/8
Subject 5 done.
Processing Subject 6/8
Subject 6 done.
Processing Subject 7/8
Subject 7 done.
Processing Subject 8/8
Subject 8 done.
Mean Accuracy (CCACSP): [0.87268519 0.7974537  0.91087963 0.64351852 0.58564815 0.9212963
 0.72453704 0.93981481]
Mean Kappa (CCACSP): [0.74537037 0.59490741 0.82175926 0.28703704 0.1712963  0.84259259
 0.44907407 0.87962963]
Mean Accuracy (Classic CSP): [0.85300926 0.79282407 0.90046296 0.69328704 0.60763889 0.91550926
 0.72222222 0.93287037]
Mean Kappa (CSP): [0.70601852 0.58564815 0.80092593 0.38657407 0.21527778 0.83101852
 0.44444444 0.86574074]
